In [ ]:
pip install rapidfuzz lifetimes matplotlib tqdm pytz

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
from rapidfuzz import process, fuzz
from tqdm import tqdm
import pytz
from lifetimes import GammaGammaFitter

import warnings
warnings.filterwarnings('ignore')

In [2]:
online_content_data = pd.read_csv('Data/XYZ_NETWORKS_ONLINE_CONTENT_CONSUMPTION.csv')
online_content_data.head()

,ID,URL,DATE,SCORE
0,028ZIWM575,https://www.vodmedia.com/target-locked-episode-10,2023-05-23T17:26:54+00:00,0.44772
1,862RYTF544,https://www.videodirect.com/stormfront-the-fin...,2022-03-03T17:32:02+00:00,NaN
2,467WUAP768,https://www.vodmedia.com/deep-waters-submarine...,2023-08-04T08:57:06+00:00,0.66931
3,765KZVX773,https://www.watchnow.com/deep-waters-submarine...,2023-01-25T12:17:42+00:00,0.88933
4,774ZQUY606,https://www.vodflix.com/target-locked-s2e5,2023-08-03T02:05:50+00:00,0.64074


In [3]:
online_content_data.shape

(799895, 4)

## Data Preprocessing

In [4]:
# drop rows with na
df = online_content_data.dropna()

In [5]:
def extract_show(url):
    path = re.sub(r'^https?://www\.[^/]+/', '', url)
    path = re.sub(r'^(movies/|shows/|series/)', '', path)
    path = re.sub(r'(-episode.*|-season.*|-s\d+e\d+.*|-part.*)$', '', path)
    show = path.replace('-', ' ').strip()
    return show if show else None

In [6]:
# append brand and show name column to df
df['Show'] = df['URL'].apply(extract_show)

In [7]:
# transform DATE to datetime object
df['DATE'] = pd.to_datetime(df['DATE'])

In [8]:
df['year'] = df['DATE'].dt.year
df['month'] = df['DATE'].dt.month
df['day'] = df['DATE'].dt.day
df['hour'] = df['DATE'].dt.hour
df['dayofweek'] = df['DATE'].dt.dayofweek+1

In [9]:
# filter out rows where 'SCORE' < 0.5

df = df[df['SCORE'] > 0.2]

In [10]:
df.head()

,ID,URL,DATE,SCORE,Show,year,month,day,hour,dayofweek
0,028ZIWM575,https://www.vodmedia.com/target-locked-episode-10,2023-05-23 17:26:54+00:00,0.44772,target locked,2023,5,23,17,2
2,467WUAP768,https://www.vodmedia.com/deep-waters-submarine...,2023-08-04 08:57:06+00:00,0.66931,deep waters submarine showdown,2023,8,4,8,5
3,765KZVX773,https://www.watchnow.com/deep-waters-submarine...,2023-01-25 12:17:42+00:00,0.88933,deep waters submarine showdown,2023,1,25,12,3
4,774ZQUY606,https://www.vodflix.com/target-locked-s2e5,2023-08-03 02:05:50+00:00,0.64074,target locked,2023,8,3,2,4
5,234FCFH350,https://www.vodmedia.com/the-last-outpost-s4e9,2023-02-05 15:24:11+00:00,0.86975,the last outpost,2023,2,5,15,7


In [11]:
df.shape

(467708, 10)

In [12]:
# map brand to each show

brand_shows = {
    'LimeLight': [
        'Fashion Frenzy: The Runway Wars',
        'Home Makeover Rescue',
        'Celebrity Chefs Showdown',
        'Style Seekers: Global Trends',
        'The Perfect Party Planner',
        'Behind the Glam: Celebrity Secrets',
        'Living Large: Luxury Homes Edition',
        'Fitness Gurus: Transformations',
        'Destination Wedding Dreams',
        'Extreme Makeovers: House Flips'
    ],
    'Pulse': [
        'Edge of Extinction',
        'Highway Heist',
        'Stormfront: The Final Mission',
        'Deep Waters: Submarine Showdown',
        'Midnight Pursuit',
        'Target Locked',
        'Chasing Shadows: Elite Unit',
        'Blood Oath: Vigilante Justice',
        'Flight Risk',
        'The Last Outpost'
    ],
    'ChillStream': [
        'Oceans Untamed: The Deep Unknown',
        'Ancient Civilizations Uncovered',
        'World Wonders: Nature\'s Marvels',
        'The Amazon Trail: A Journey Through the Rainforest',
        'Surviving Antarctica: The Final Frontier',
        'Wildlife Warriors: Protectors of the Endangered',
        'Unseen Worlds: Microscopic Marvels',
        'Nomads of the North: Life in the Arctic',
        'Mount Everest: Beyond the Summit',
        'Wonders of the Coral Reef'
    ],
    'RetroReel': [
        'Vintage Noir: Tales of the 50s',
        'Hollywood Gold: The Best of the Silver Screen',
        'The Golden Age of Television',
        'Classic Cartoons Rewind',
        'Heroes of the West: Old Cowboy Adventures',
        'The Big Picture: Cinema’s Finest Years',
        'Love in Black and White: Romantic Classics',
        'Retro Sci-Fi: Space and Beyond',
        'Timeless Thrillers: Hitchcock’s Legacy',
        '1950s Sitcom Showcase'
    ],
    'CineQuest': [
        'The Forgotten Kingdom',
        'Unwritten Laws',
        'Echoes of the Past',
        'Shattered Mirror: A Tale of Betrayal',
        'Lies Beneath the Surface',
        'After the Storm: A Family’s Battle',
        'In the Shadow of Giants',
        'Fallen Crown: The Battle for the Throne',
        'Last Sunset: A Dystopian Love Story',
        'Whispers in the Dark'
    ],
    'Adrenaline': [
        'Chopper Chase: Ride to Freedom',
        'Survivalist: Desert Run',
        'The High Stakes Heist',
        'Crash Course: Daredevil Driving',
        'Outrun the Law: Speed Trials',
        'Extreme Elevation: Mountain Mayhem',
        'Race Against Time: Urban Parkour',
        'Rogue Waves: Ocean Survival',
        'Battle on the Bridge',
        'Canyon Plunge: The Ultimate Cliff Dive'
    ],
    'DarkMatter': [
        'Mind Games: The Final Puzzle',
        'The Silent Watcher',
        'Vanishing Point: The Missing Files',
        'Twisted Shadows',
        'Behind Closed Doors: A Psychological Nightmare',
        'Double Crossed: The Deep Conspiracy',
        'The Last Witness',
        'False Alibis',
        'The Confession That Never Was',
        'Voices in the Dark'
    ],
    'SilverScreen Classics': [
        'Midnight Masquerade',
        'The Gunslinger’s Code',
        'Dancing in the Rain: A Musical Romance',
        'Secrets of the Starlet',
        'Heartstrings: A Love to Remember',
        'Man on the Moon: A Sci-Fi Classic',
        'Western Trails: The Lone Ranger',
        'Whispers in the Night',
        'The Jazz Club: Music and Romance',
        'Beneath the Scarlet Veil'
    ],
    'TimeCapsule TV': [
        'The Brady Family Reunion',
        'Golden Laughs: Classic Sitcoms',
        'Super Sleuth: Detective Rewind',
        'Game Show Fever: Retro Revival',
        'The Wonder Years: Growing Up in the 80s',
        'The Mid-Century Mystery Hour',
        'Our Neighborhood: Family Stories from the 70s',
        'Classic Comedy Club',
        'The Great Quiz Show Showdown',
        'The Retro Detective: Case Files Reopened'
    ],
    'TasteMakers': [
        'The Chef Showdown: Battle of the Kitchens',
        'Food Truck Frenzy',
        'Global Gourmet: Culinary Journeys',
        'Bake Off Bonanza',
        'Flavors of the World: Street Eats',
        'Top Chef All-Stars: The Final Feast',
        'Sugar Rush: Desserts on Deadline',
        'Master Butcher: Carving the Perfect Cut',
        'Spice Wars: The Heat is On',
        'Farm to Table: The Chef’s Challenge'
    ],
    'DesignLab': [
        'Renovation Rescue: Extreme Makeovers',
        'Modern Marvels: Futuristic Home Designs',
        'Tiny Homes, Big Dreams',
        'Design Masters: Showdown of the Styles',
        'Budget Renovation: Luxe for Less',
        'Curb Appeal: Outdoor Transformations',
        'Dream Spaces: Designer’s Pick',
        'Flip or Flop: The Ultimate Remodel',
        'Eco-Living: Sustainable Homes',
        'Minimalist Dreams: Designing with Less'
    ],
    'PopCulture Now': [
        'Star Style: Hollywood’s Fashion Icons',
        'The Influencer Effect: Social Media Superstars',
        'Celebrity Gossip Confidential',
        'Red Carpet Rundown: Awards Show Style',
        'Hot List: Who’s Trending Now?',
        'Fashion Faux Pas: Hits and Misses',
        'Inside the A-List: Celebrity Homes',
        'The Insta-Celeb Diaries',
        'Behind the Fame: The Rise of Pop Culture Icons',
        'Entertainment Flash: The Week in Review'
    ]
}

In [13]:
# show to brand mapping

show_to_brand = {}
for brand, shows in brand_shows.items():
    for show in shows:
        show_to_brand[show.lower()] = brand

In [14]:
all_show_names = list(show_to_brand.keys())

In [15]:
# fuzzy match function

def match_show_name(show_name, choices, scorer=fuzz.WRatio, cutoff=80):
    """
    Matches the show_name to the closest show in choices using fuzzy matching.
    Returns the matched show name if the score is above the cutoff; otherwise, returns None.
    """
    if pd.isnull(show_name):
        return None
    show_name = show_name.lower()
    match = process.extractOne(show_name, choices, scorer=scorer)
    if match and match[1] >= cutoff:
        return match[0]
    else:
        return None

In [16]:
tqdm.pandas()

df['Matched_Show'] = df['Show'].progress_apply(lambda x: match_show_name(x, all_show_names))

100%|██████████| 467708/467708 [01:30<00:00, 5193.79it/s]


In [17]:
df[df['Matched_Show'].isna()]

,ID,URL,DATE,SCORE,Show,year,month,day,hour,dayofweek,Matched_Show
22431,725XTDH624,Here are 100 more URLs following your specifie...,2022-07-22 12:12:22+00:00,0.26150,Here are 100 more URLs following your specifie...,2022,7,22,12,5,None
22861,321DQGU207,Here are 100 more URLs following your specifie...,2022-08-26 23:17:19+00:00,0.72855,Here are 100 more URLs following your specifie...,2022,8,26,23,5,None
22887,859GXFC303,Here are 100 more URLs following your specifie...,2022-11-11 09:29:24+00:00,0.80164,Here are 100 more URLs following your specifie...,2022,11,11,9,5,None
22946,033OCLA420,Here are 100 more URLs following your specifie...,2022-03-26 05:35:49+00:00,0.27399,Here are 100 more URLs following your specifie...,2022,3,26,5,6,None
23499,454DREA418,Here are 100 more URLs following your specifie...,2022-11-25 17:21:14+00:00,0.54828,Here are 100 more URLs following your specifie...,2022,11,25,17,5,None
...,...,...,...,...,...,...,...,...,...,...,...
770377,965OZYV564,Here are 100 more URLs following your specifie...,2023-01-04 20:56:34+00:00,0.95549,Here are 100 more URLs following your specifie...,2023,1,4,20,3,None
770670,989XJRY624,Here are 100 more URLs following your specifie...,2023-03-14 19:21:24+00:00,0.22902,Here are 100 more URLs following your specifie...,2023,3,14,19,2,None
770844,796FGZK586,Here are 100 more URLs following your specifie...,2022-05-16 04:01:01+00:00,0.26654,Here are 100 more URLs following your specifie...,2022,5,16,4,1,None
770867,687VVTG679,Here are 100 more URLs following your specifie...,2023-07-26 19:50:18+00:00,0.84241,Here are 100 more URLs following your specifie...,2023,7,26,19,3,None


In [18]:
# drop invalid URL format rows

df = df[~df['Matched_Show'].isna()]

In [19]:
# map each row to a brand

df['Brand'] = df['Matched_Show'].map(show_to_brand)

In [20]:
df['Brand'].isna().sum()

np.int64(0)

In [21]:
df.head()

,ID,URL,DATE,SCORE,Show,year,month,day,hour,dayofweek,Matched_Show,Brand
0,028ZIWM575,https://www.vodmedia.com/target-locked-episode-10,2023-05-23 17:26:54+00:00,0.44772,target locked,2023,5,23,17,2,target locked,Pulse
2,467WUAP768,https://www.vodmedia.com/deep-waters-submarine...,2023-08-04 08:57:06+00:00,0.66931,deep waters submarine showdown,2023,8,4,8,5,deep waters: submarine showdown,Pulse
3,765KZVX773,https://www.watchnow.com/deep-waters-submarine...,2023-01-25 12:17:42+00:00,0.88933,deep waters submarine showdown,2023,1,25,12,3,deep waters: submarine showdown,Pulse
4,774ZQUY606,https://www.vodflix.com/target-locked-s2e5,2023-08-03 02:05:50+00:00,0.64074,target locked,2023,8,3,2,4,target locked,Pulse
5,234FCFH350,https://www.vodmedia.com/the-last-outpost-s4e9,2023-02-05 15:24:11+00:00,0.86975,the last outpost,2023,2,5,15,7,the last outpost,Pulse


In [22]:
# count of user visiting each brand

df.groupby('Brand')['ID'].nunique().sort_values(ascending=False)

Brand
LimeLight                18929
SilverScreen Classics    13502
TimeCapsule TV           13501
DarkMatter               13030
Adrenaline               12891
PopCulture Now           12818
DesignLab                12809
TasteMakers              12710
RetroReel                12132
Pulse                    10854
CineQuest                 9884
ChillStream               8513
Name: ID, dtype: int64

In [23]:
df.groupby('Brand')['SCORE'].mean().sort_values(ascending=False)

Brand
TasteMakers              0.601952
DesignLab                0.601341
PopCulture Now           0.601105
Pulse                    0.600751
ChillStream              0.600422
CineQuest                0.600260
RetroReel                0.599981
SilverScreen Classics    0.599219
TimeCapsule TV           0.599005
Adrenaline               0.598636
LimeLight                0.598481
DarkMatter               0.597958
Name: SCORE, dtype: float64

## Train Test Split

train:   X (2022-01 to 2022-12)  y (2023-01 to 2023-06)

test:    X (2022-06 to 2023-06)  y (2023-06 to 2023-12)

predict: X (2023-01 to 2023-12)  y (2024-01 to 2024-06)

In [27]:
train_start, train_end = '2022-01-01', '2022-12-31'
test_start, test_end = '2022-06-01', '2023-06-30'
predict_start, predict_end = '2023-01-01', '2023-12-31'

In [28]:
# Train set: X from 2022-01 to 2022-12, y from 2023-01 to 2023-06
train_X = df[(df['DATE'] >= train_start) & (df['DATE'] <= train_end)]
train_y = df[(df['DATE'] > '2022-12-31') & (df['DATE'] <= '2023-06-30')]

In [29]:
# Test set: X from 2022-06 to 2023-06, y from 2023-06 to 2023-12
test_X = df[(df['DATE'] >= test_start) & (df['DATE'] <= test_end)]
test_y = df[(df['DATE'] > '2023-06-30') & (df['DATE'] <= '2023-12-31')]

In [30]:
# Prediction set: X from 2023-01 to 2023-12, y from 2024-01 to 2024-06
predict_X = df[(df['DATE'] >= predict_start) & (df['DATE'] <= predict_end)]
predict_y = None # future data: 6 months

In [31]:
print("Train X shape:", train_X.shape)
print("Train y shape:", train_y.shape)
print("Test X shape:", test_X.shape)
print("Test y shape:", test_y.shape)
print("Predict X shape:", predict_X.shape)

Train X shape: (233412, 12)
Train y shape: (115385, 12)
Test X shape: (251593, 12)
Test y shape: (117698, 12)
Predict X shape: (232459, 12)


## Calculate Recency, Frequency, and Tenure

In [40]:
def calculate_frequency_tenure(df):
    df_grouped = df.groupby(['ID', 'Brand']).agg({
        'SCORE': 'sum',
        'DATE': ['min', 'max']
    }).reset_index()

    df_grouped.columns = ['ID', 'Brand', 'score_sum', 'first_visit_date', 'last_visit_date']

    # Calculate Frequency: total number of visits to each brand (rounded to the nearest integer)
    df_grouped['frequency'] = df_grouped['score_sum'].round().astype(int)

    # Calculate Tenure: time beteen the first and last visit
    df_grouped['tenure'] = (df_grouped['last_visit_date'] - df_grouped['first_visit_date']).dt.days

    # Calculate Recency: time since the customer's last visit

    reference_date = df['DATE'].max()
    df_grouped['recency'] = (reference_date - df_grouped['last_visit_date']).dt.days

    return df_grouped

In [ ]:
train_X_grouped = calculate_frequency_tenure(train_X)
train_y_grouped = calculate_frequency_tenure(train_y)

test_X_grouped = calculate_frequency_tenure(test_X)
test_y_grouped = calculate_frequency_tenure(test_y)

predict_X_grouped = calculate_frequency_tenure(predict_X)

In [44]:
train_X_grouped.to_csv('Grouped/train_X_grouped.csv')
train_y_grouped.to_csv('Grouped/train_y_grouped.csv')

test_X_grouped.to_csv('Grouped/test_X_grouped.csv')
test_y_grouped.to_csv('Grouped/test_y_grouped.csv')

predict_X_grouped.to_csv('Grouped/predict_X_grouped.csv')

## Combine with Revenue

In [43]:
revenue_df = pd.read_csv('data_preprocessing/sub_duration_revenue.csv')
revenue_df.head()

,Unnamed: 0,ID,BRAND,SUB_DETAIL,START_DATE,END_DATE,SUB_DURATION_DAYS,REVENUE
0,0,000AHHU956,RetroReel,expired monthly,2022-01-01,2022-02-17,47,60.0
1,1,000AHHU956,RetroReel,paid monthly,2022-05-21,2023-10-28,525,600.0
2,2,000BRHY264,CineQuest,paid annual,2022-05-23,2023-10-17,512,120.0
3,3,000DJKK225,CineQuest,expired monthly,2022-01-01,2022-04-15,104,48.0
4,4,000DTPC747,LimeLight,expired annual,2022-01-01,2022-08-01,212,80.0


## Gamma-Gamma Model

In [ ]:
ggf = GammaGammaFitter(penalizer_coef=0.0)

ggf.fit(
    frequency = train_X_grouped['frequency'],
    monetary_value = df_grouped['monetary_value']
)

KeyError: 'monetary_value'

## Calculate CLV

In [ ]:
df_grouped['CLV'] = ggf.customer_lifetime_value(
    bgf,
    frequency = df_grouped['frequency'],
    recency = df_grouped['recency'],
    T = df_grouped['tenure'],
    monetary_value = df_grouped['monetary_value'],
    time = 6,  # months
    freq = 'D',  # 'D' for daily frequency
    discount_rate = 0.01  # Monthly discount rate (~12% annually)
)

2022-01-2022-12 train, 2022-06-01-2023-06-01 valid, 2023-01-2023-12 predict

y revenue 2023-01-2023-06 train; 2023-06-2023-12 valid, predict next 6 months